In [1]:
import pandas as pd
import numpy as np
import random as rd
import warnings, glob, os, sys, shutil
from tqdm.notebook import tqdm
pd.set_option("display.max_columns", None)
warnings.filterwarnings("ignore")

In [ ]:
import numpy

In [2]:
# 過濾非生理數據與未使用特徵
drop_cols = ['洗腎紀錄時間', '急診門診住院', '生日', '算年齡', 
             'DLC', 'PERMCATH', 'HBs', 'HBsAg', 'HBc', 
             
             '體重2標準體重', '體重3應脫水', '體重結束', '分母體重', '體重機器顯示', 
             'BUN (洗腎後)', 
             'sbp', 'dbp', 'New Intra SBP', 'New Intra DBP',
             'New 處置其他+症狀處置', 'New 處置其他結束', '處置其他結束', '處置其他', '症狀處置', 'NLP Judge', 'BP Judge']

select_cols =  ['性別', '入院方式', 'HCV','HBV','有無糖尿病','FISTULA','GRAFT','Catheter','Intact PTH','age', '體重1開始','開始血壓SBP', '開始血壓DBP',
            '開始脈搏', '體溫', '體重實際脫水','每公斤脫水量(ml/kg)','BUN','K', 'HGB','URR%','Na', 'Ca','P',
            '透析液 Ca','ALBUMIN','ALT (SGPT)','Alk.phosphatase','Ferritin','IRON/TIBC','MCV', 'MCHC', 'MCH','Iron','Glucose AC','RBC', 'WBC',
            'Platelet', 'Creatinine','AST (SGOT)','TIBC','Bilirubin-T', 'Cholesterol-T', 'CRP']
select_cols = select_cols+ ['Max Diff mbp', 'Max Diff sbp','結束脈搏','Final Judge','Raw Index','ID','洗腎紀錄時間去時分',
                            'fold_0','fold_1', 'fold_2', 'fold_3', 'fold_4']

mask_cols = ['Max Diff mbp','Max Diff sbp', 'Final Judge','結束脈搏'] #'每公斤脫水量(ml/kg)'

In [3]:
# CV process 
ths_days = 30 #change 30 => 90
stack_nums = 12 #max = 12 / 16 lv.3=16, 8, 4
input_nums = 12

final_cv_save_path = './dataset/TimeSeries_TD{}_SN{}_IN{}_CV_[2024-09-30]'.format(ths_days, stack_nums, input_nums)
os.makedirs(final_cv_save_path, exist_ok=True) #創建儲存路徑

def data_ts_stack(group_data_type, group_ID_df, columns, ts_save_path):
    for idx__, (group_name, group_data) in enumerate(tqdm(group_ID_df)):
        timeseries_df = pd.DataFrame([], columns=columns)
        timeseries_df['Group Type'] = np.nan #標記Train、Valid、Test
        group_data = group_data.sort_values(by=['洗腎紀錄時間去時分'], ascending=True)
        group_data = group_data.reset_index(drop=True)
        for idx in group_data.index:
            slice_df = group_data.loc[idx:idx+(stack_nums-1)].copy()
            temp_df = slice_df
            if slice_df.shape[0]==stack_nums and (abs(slice_df['洗腎紀錄時間去時分'].iloc[0] - slice_df['洗腎紀錄時間去時分'].iloc[-1]).days<=ths_days): #判斷資料是否有{stack_nums}筆，以及區間時間是否小於等於{ths_days}
                # print(slice_df.shape)
                if stack_nums!=input_nums:
                    slice_df = slice_df.iloc[(stack_nums-input_nums)::]
                    slice_df = pd.concat([slice_df,slice_df.iloc[-1].to_frame().T.copy()])
                else: 
                    slice_df = pd.concat([slice_df,slice_df.iloc[-1].to_frame().T.copy()])
                slice_df.iloc[-2,slice_df.columns.get_indexer(mask_cols)] = -9999
                timeseries_df = pd.concat([timeseries_df,slice_df])
        if timeseries_df.shape[0]!=0:
            timeseries_df.to_csv(os.path.join(ts_save_path,f'{group_data_type}-{str(group_name[0])}.csv'),encoding='utf-8-sig', index=False)
        # break

In [4]:
Data_ = {
    'A':'./dataset/DATA_A_FinalFinished[2024-09-27-filter-age]fixed.csv',
    'B':'./dataset/DATA_B_FinalFinished[2024-09-27-filter-age]fixed.csv',
    'C':'./dataset/DATA_C_FinalFinished[2024-09-27-filter-age]fixed.csv'
    }

In [5]:
for DATA_NAME in Data_:
    filter_data_df = pd.read_csv(Data_[DATA_NAME])
    filter_data_df = filter_data_df[select_cols]
    filter_data_df['洗腎紀錄時間去時分'] = pd.to_datetime(filter_data_df['洗腎紀錄時間去時分']).dt.date
    
    # for cv_fold in ['fold_0','fold_1', 'fold_2', 'fold_3', 'fold_4']:
    ts_save_path = './dataset/CV_temp/'
    if os.path.exists(ts_save_path):
        shutil.rmtree(ts_save_path)
    os.makedirs(ts_save_path, exist_ok=True) #創建儲存路徑
    # train_df = filter_data_df[filter_data_df[cv_fold]=='train']
    # group_ID_train_df = train_df.groupby(['ID'])
    # data_ts_stack('train', group_ID_train_df, train_df.columns, ts_save_path)
    # valid_df = filter_data_df[filter_data_df[cv_fold]=='val']
    # group_ID_valid_df = valid_df.groupby(['ID'])
    # data_ts_stack('valid', group_ID_valid_df, valid_df.columns, ts_save_path)
    group_ID_filter_data_df = filter_data_df.groupby(['ID'])
    data_ts_stack('TrainVal', group_ID_filter_data_df, filter_data_df.columns, ts_save_path)
    
    patient_csv_glob = glob.glob(os.path.join(ts_save_path,'*'))
    # trainval_df = pd.DataFrame([], columns=valid_df.columns)
    trainval_df = pd.DataFrame([], columns=filter_data_df.columns)
    for csv_path in tqdm(patient_csv_glob):
        data_df = pd.read_csv(csv_path)
        trainval_df = pd.concat([trainval_df,data_df])
        del data_df
    # save_name = "TrainVal_{}_DATA-{}.csv".format(cv_fold, DATA_NAME)
    save_name = "TrainVal_DATA-{}.csv".format( DATA_NAME)
    trainval_df.to_csv(os.path.join(final_cv_save_path, save_name), encoding='utf-8-sig', index=False)    
    del trainval_df


  0%|          | 0/774 [00:00<?, ?it/s]

  0%|          | 0/574 [00:00<?, ?it/s]

  0%|          | 0/773 [00:00<?, ?it/s]

  0%|          | 0/573 [00:00<?, ?it/s]

  0%|          | 0/508 [00:00<?, ?it/s]

  0%|          | 0/478 [00:00<?, ?it/s]